<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/Ex03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math

url = "https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/Antibiotic_70K_patinets.csv"
data = pd.read_csv(url)

### [1]

In [2]:
data.describe()

,ID,total_d,high_d,low_d,none_high_d,high_p,cdiff,age_5
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.00000
mean,35000.500000,8.049771,3.774386,6.131871,4.275386,42.254031,0.021471,2.74870
std,20207.403759,13.885638,8.795789,11.502711,7.950063,39.836986,0.144951,1.56102
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000
25%,17500.750000,2.000000,0.000000,2.000000,1.000000,0.000000,0.000000,1.00000
50%,35000.500000,4.000000,2.000000,3.000000,2.000000,37.500000,0.000000,3.00000
75%,52500.250000,8.000000,4.000000,6.000000,4.000000,80.000000,0.000000,4.00000
max,70000.000000,526.000000,368.000000,397.000000,294.000000,100.000000,1.000000,7.00000


### [2]

In [3]:
d2 = data.groupby("age_5").agg({"high_p": "mean"}).sort_values(by="high_p", ascending=False)

print(f"{d2.index[0]}")

7


### [3]

In [4]:
print(f"{data.isna().sum().sum()}")

0


In [5]:
upper = data["total_d"].mean() + 3 * data["total_d"].std()
lower = data["total_d"].mean() - 3 * data["total_d"].std()
d3 = data.assign(outlier=data["total_d"].apply(lambda x: 1 if (x > upper) | (x < lower) else 0))

print(f"{d3[d3['outlier'] >= 1]['ID'].nunique()}")

1258


### [4]

In [6]:
d4 = data[['age_5', 'high_p']].corr(method='pearson')

print(f"{math.floor(d4.iloc[0, 1] * 100) / 100}")

0.12


### [5]

In [7]:
cd_y = data[data['cdiff'] == 1]
cd_n = data[data['cdiff'] == 0]

print(f"{math.floor(abs(cd_y['high_p'].mean() - cd_n['high_p'].mean()) * 100) / 100}")

4.79


In [8]:
from scipy.stats import ttest_ind

p_val, t_val = ttest_ind(cd_y['high_p'], cd_n['high_p'])

print(f"p : {p_val}, t : {t_val}")
print("t-value가 0.05보다 작아 귀무가설을 기각, 즉 평균이 같다고 할 수 없다")

p : 4.613035361614422, t : 3.975323449530875e-06
t-value가 0.05보다 작아 귀무가설을 기각, 즉 평균이 같다고 할 수 없다


### [6]

In [34]:
X_cols = ['total_d', 'high_d', 'low_d', 'high_p', 'age_5']
y_var = 'cdiff'

train_X = data[X_cols]
train_y = data[y_var]

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100000, random_state=1234, penalty='l2', solver='newton-cg')
model = lr.fit(train_X, train_y)

idx = [i for i, coef in enumerate(model.coef_[0]) if coef < 0]
minus_val = ", ".join([X_cols[i] for i in idx])

print(f"{minus_val}")

high_d, low_d
